# Activities

The goal of this practicum is to implement a web application programming interface (web API) that is RESTful. The Web API will use the `model`, `controller` and `action` concepts from the MVC pattern.

In previous practicums you were able to implement the `model` part and design queries that store and retrieve data from the database. In this practicum you will perform three activities. In the first activity you will implement the controller that will be mapped directly to a resources `uri` (/api/movies). Actions are used to map to specific `HTTP` verbs, such as `GET` and `POST`, to perform a specific operation (`GetAll` Movies) and to provide a response to a client. In the second activity we will add pagination functionality to our implementation. In the third activity we will add dynamic filtering using `LINQ Expressions` to our implementation.   
The activities performed are:
- 1 - Build a single RESTful controller against an EF database
- 2 - Add pagination to `GetAll`
- 3 - Add generic predicate filtering to `GetAll`

### 1 Building a RESTful controller and testing it with POSTMAN

#### Step 1: Setup project
- To start the implementation you need two addtional tools: 
    - A HTTP client named `Postman` that sends and receives HTTP messages (json) to and from a web API. Download Postman from https://www.getpostman.com/ 

    - A .net core package called `Newtonsoft.Json` to serialize json messages and deserialize .NET objects. This package will be added after project creation.


- In order to automatically generate a controller in dotnet core we will create a `webapi-application`.  Then we add the `model` part to it (see step 3). To set up a webapi-project in a folder called lesson3 execute the following command: 
```bash
dotnet new webapi -o lesson3
```
Navigate to the folder of your project and type the following commands in the command line:
```bash
dotnet add package Newtonsoft.Json
dotnet restore
```
You can compile and run this project to check if the web API is working by default. After executing the run-command `dotnet run` in the console, your web API will start working by showing the message: 

`Now listening on: http://localhost:5000`
`Application started. Press Ctrl+C to shut down.`


####  Step 2: Web API check

- By default the .net core will generate a controller for you in the root folder. Go to the Controllers folder and open the file `ValuesController.cs`. 

```csharp
using System;
using System.Collections.Generic;
using System.Linq;
using System.Threading.Tasks;
using Microsoft.AspNetCore.Mvc;

namespace lesson3.Controllers
{
    //This is the default route of the API. 
    [Route("api/[controller]")]
    public class ValuesController : Controller
    {
        // GET api/values
        [HttpGet]
        public IEnumerable<string> Get()
        {
            return new string[] { "value1", "value2" };
        }

        // GET api/values/5
        [HttpGet("{id}")]
        public string Get(int id)
        {
            return "value";
        }

        // POST api/values
        [HttpPost]
        public void Post([FromBody]string value)
        {
        }

        // PUT api/values/5
        [HttpPut("{id}")]
        public void Put(int id, [FromBody]string value)
        {
        }

        // DELETE api/values/5
        [HttpDelete("{id}")]
        public void Delete(int id)
        {
        }
    }
}
```

- By default the web API will have the uri: `http://localhost:5000/api/values`. To check the communication between this API and a client we will use `Postman`. After starting `Postman`, select the command `GET`, enter the uri mentioned above and click on the `send` button. Remember also to turn off the SSL certificate verification (from `settings`). After a few seconds, you should receive a HTTP message from your api in json-format. The default message should be: 

`{"value1", "value2"}`.
 
#### Step 3: Add a model

Now we need to adapt the default controller (ValuesController.cs) to get it working with our data storage and the model from practicum1. 

- You need to make few adjustments to get that model working in your current project setup. 
    - Create a folder named `Models`
    - Copy the `Model.cs` file from your previous project (practicum1) to the `Models` folder of your current project. 
    - Change the namespace to match the following:
```csharp
namespace lesson3.Models
```
- The files that need to be adjusted from now on are: 
    - the .csproj file in the root directory of your project
    - `Model.cs` in the Models folder
    - `Startup.cs` in the root directory of your project
    - `ValuesController.cs` in the Controllers folder 
 
- Open the .csproj file of this project and copy into it the itemgroups related to the EntityFrameworkCore from the .csproj file of the project of the previous practicum.
- Change the class `MovieContext` in `Model.cs` to match this structure: 

```csharp
...
namespace lesson3.Models
{
 public class MovieContext : DbContext
    {

        public DbSet<Actors> Actors { get; set; }
        public DbSet<Movies> Movies { get; set; }
        
        //Added constructor to provide the connection to the database as a service (look at: startup.cs)
        public MovieContext(DbContextOptions<MovieContext> options): base(options)
        {
        }
        
        //Uncomment or remove the OnConfiguring method. It is not needed any more
        // protected override void OnConfiguring(DbContextOptionsBuilder optionsBuilder)
        // {
        //     if (!optionsBuilder.IsConfigured)
        //     {
        //
        //         // http://go.microsoft.com/fwlink/?LinkId=723263 for guidance on storing connection strings
        //         optionsBuilder.UseNpgsql(@"Host=localhost;Database=MovieDB;Username=postgres;Password=postgres");
        //     }
        // }
...
}
```

- The connection to the database will be provided as a service in this project. You need to add the connection string  to the `ConfigureServices` method in `Startup.cs`. Remember to change the connection parameters.

```csharp
...

// This method gets called by the runtime. Use this method to add services to the container.
public void ConfigureServices (IServiceCollection services) {

        //Add this line to your method
     services.AddDbContext<MovieContext> (
                 opt => opt.UseNpgsql(@"Host=localhost;Database=MovieDB;Username=postgres;Password=postgres"));
     
     services.AddMvc ();

        }
...

```

- Change the name `ValuesController.cs` to `MoviesController.cs`.  We need to add a private property for the context to access it from our controller. This property will be initilized in the contructor. To understand how the context can be used in the controller the method `Get()` and `Get(int id)` are implemented to get the movies as a result form the database. 

```csharp

using System;
using System.Collections.Generic;
using System.Linq;
using System.Linq.Expressions;
using System.Threading.Tasks;
using lesson3.Models;
using Microsoft.AspNetCore.Mvc;

namespace lesson3.Controllers {

    [Route ("api/[controller]")]
    public class MoviesController : Controller {
        private readonly MovieContext _context;

        public MoviesController (MovieContext context) {
            _context = context;
        }

        // GET api/movies
        [HttpGet]
        public IEnumerable<Movies> Get () {
            return _context.Movies.ToList ();
        }

        // GET api/movies/5
        [HttpGet ("{id}")]
        public IActionResult Get (int id) {
            var movie = _context.Movies.FirstOrDefault (t => t.MovieId == id);
            if (movie == null) {
                return NotFound ();
            }
            return new ObjectResult (movie);
        }

        // POST api/movies
        [HttpPost]
        public void Post ([FromBody] Movies value) { }

        // PUT api/movies/5
        [HttpPut ("{id}")]
        public void Put (int id, [FromBody] Movies value) { }

        // DELETE api/movies/5
        [HttpDelete ("{id}")]
        public void Delete (int id) { }
    }
}

```
- Restart your server by first typing `Ctrl+C` and then typing the command `dotnet run`. 

### 2 Adding paginations

Pagination comes useful when dealing with big amount of data. Since it is too _expensive_ to move big abmount of data between client and server, we use pagination as a mean to load the data resulting from a query, but instead of loading them all at once we load them in pages.

In the following schema a user is asking for the page at position 3 of size 3. The controller will use the given page size info to split the data into pages, each containing at most 3 items, and returning the page in position 3 (in our case the fourth one).

Together with the page's items the controller returns also the total number of pages; this is necessary in case the client needs to know how many pages are left to visit all the items.

<img src="20.jpg">

To implement this solution we need first to create our page data structure, which needs to be copied both in the client (in case you are using a http client in C#) and in the server.

<img src="17.JPG">

Then we extend the DbSet data structure with a generic extension method that allows the pagination of the result of a query.

<img src="16.JPG">

Once the extension method is defined, we can use it in any query that requires pagination. In our case we use it inside a _get method_ that selects all actors from our database.
<img src="19.JPG">

Remember to include the paginator and extension method namespace in controller's namespace.
<img src="18.JPG">

Now, test the just made code with postman.
<img src="21.JPG">

As an exercise, try to use pagination with more complex queries (like select all actors that are older than 31) and test them with postman.


### 3 Generic predicate filtering
To use generic predicates to select entities of our database you need to use LINQ dynamically by means of `Expression Trees` [1]. Expression trees allow to build/compose dynamically LINQ objects that can be used for testing (example whether to select or not an entity, to order the result of a query, etc.).



In the following picture we present a schematized version of a simple application. In the schema a client is trying to get all the actors from a remote database that are called `Pippo` and which ages are 18. To do so the client first builds a filter expression (**1**) that captures the just mentioned conditions, and then calls (**2**) the remote method `GetAll` to which the we pass the filter expression (_our predicate_) as a json string. 



The controller captures the call request (**3**) and forwards it to the right method (**4**). The method `GetAll` converts the input filter expression to an equivalent version in `Expression Tree`. The converted expression and the rest of the query is then passed to EF (**5**) which takes care of selecting the right actors. Once selected, the actors are returned back to the client (**6**). The client, which in the meantime was waiting for a response (**2.1**), receives the actors and saves them (**7**).



<img src="1.jpg">



In the following subsections the code for the client and the server is provided and commented. Use the above image to follow the structure of the solution.

#### Client
For this solution we use a client written in C# (you can skip this part and use directly postman; in this case make sure to build correctly the body of the predicate to send via http). 


Create a console application `dotnet new console` and add the Newtonsoft package to it `dotnet add package Newtonsoft.Json`. Newtonsoft is needed to parse the predicate into a json structure to send to the server. In the end the code should look like the following:


<img src="2.JPG">

The classes needed for the correct functioning of the code above are:

<img src="3.JPG">




Before you run the program make sure that the server (which definition is provided in the following subsection) is running.

#### Server

To implement the server we need to make a _webapi_ project (`dotnet new webapi`). Once created, add to it the design package (`dotnet add package Microsoft.EntityFrameworkCore.Design`), and the sqlite package (`dotnet add package Microsoft.EntityFrameworkCore.Sqlite`), and by hand add a reference to the _EntityFrameworkCore_ in the projects' settings file (the one that ends in *.csproj).


In the end the project settings file should look like this:

<img src="4.JPG">

Once created, make a folder _Models_ and add to it the models from the previous practicums. You need to change for this implementaiton the constructor of the context. It should eventually look like the following (in this example we use the 1<Movie>-N<Actors> example):


<img src="5.JPG">

Modify the Startup.cs file to connect our model to the controller.

<img src="6.JPG">


In the _Controllers_ folder add a MovieController.cs file. In this file we will configure the controller, which will take care of providing our movies and actors to the client.

In the just made MovieController file add the folowing usings:

<img src="7.JPG">

Copy the _Filter_ class from the clients' code. We need this class because the deserializer in our server needs to know how to build-back the seriealized predicate. 


<img src="8.JPG">

Add a MovieController class and a contructor to it that takes care of saving locally the database context, and filling the database with some initial values (for debugging).

<img src="9.JPG">

Add now the controller that will return the actors that satisfy a certain predicate.

<img src="10.JPG">

To conclude, we implement the method that converts our filter predicate to an expression tree.


<img src="11.JPG">

To run the server use the command `dotnet run`. Now test it by calling `run` on the client written above.


References:
- [1] https://docs.microsoft.com/en-us/dotnet/csharp/programming-guide/concepts/expression-trees/
